## Exercise (Intermediate Machine Learning: 2 of 7)
### 목차:
- 결측치, 범주형 데이터
- 머신러닝 Pipeline 구축
- cross-validation (더 발전된 모델 유효성 검사)
- XGBoost (대회에서 가장 널리 쓰이는 최신 모델)
- leakage (가장 많이 하는 실수)

## Exercise (Intermediate Machine Learning: 2 of 7)

### Missing Values
결측치가 있는 데이터를 가지고 model을 만들려고 하면 error를 발생시키므로 이를 방지하기 위해선 아래 3가지 방법으로 해결할 수 있다.
1. (Simple option) 결측치가 있는 column 제거
2. (Better option) 각 열의 mean같은 수치로 채워 넣기 [Imputation=결측값 대체]
3. 2번째 방법을 사용하되 결측치 였다는 것을 알 수 있는 새로운 열 추가

대체로 1번 방법보다 2, 3번 방법으로 변환시킨 데이터로 사용한 모델의 성능이 좋게 나온다.  
그 이유는 drop column한다는 것이 결국 유용한 정보를 제거한다는 것이기 때문이다. 
```python
# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])
```

※절대 임의적으로 0으로 값을 채워넣지 말 것!

※ simpleImputer의 strategy option에는 ['mean', 'median', 'most_frequent', 'constant']가 있다.

In [ ]:
## 1. Simple Approach for Missing values
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

In [ ]:
## 2. Imputation Approach for Missing values
from sklearn.impute import SimpleImputer

# Imputation (결측값 대체)
my_imputer = SimpleImputer() # (default) stratge='mean'
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))
# fit(): some calculation (ex. means of columns from data)
# transform(): transformation (ex. replacing the missing value)
## fit_transform() → for training set
## transform() → for test/validation set (training 기반으로 학습[계산]된 모델을 사용 하므로)
# https://www.kaggle.com/questions-and-answers/58368


# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

In [ ]:
## 3. An Extension to Imputation Approach for Missing values
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

## Exercise (Intermediate Machine Learning: 3 of 7)

### Categorical Variables
설문조사 결과에 대한 데이터는 제한된 보기에 대한 설문과 열린 보기에 대한 설문이 있다. 이는 모두 **범주형 데이터(Categorical Variables)**이다.

이런 범주형 데이터를 처리하는 방법은 3가지가 있다.:
1. 범주형 변수가 속한 데이터 제거 (권장하지 않음)
2. Label Encoding
3. One-Hot Encoding

Label Encoding은 변수에 해당되는 데이터가 순서가 있을 때 유용하다. (Ordinal Variables) 예를 들어, Never < Rarely < Mostly < Everyday와 같이 랭크를 매길 수 있을 때 좋다.  
One-Hot Encoding은 데이터에 순서가 없는 변수에 유용하다. (Nominal Variabels) 하지만 보통 15개 이상의 경우의 수를 가지고 있는 경우에 모델 속도를 저하시킨다는 단점이 있다.

```python
# 수치형이 아닌 범주형 데이터를 조회하는 방법
# Get list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)
```

※ [pandas.DataFrame.select_dtypes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.select_dtypes.html) → Parameters: include, exclude

Label Encoding 이용시 발생할 수 있는 문제: <u>train에서는 발견되지 않은 데이터에 대하여 labeling 할 수 없음.</u>
```python
# All categorical columns
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# Columns that can be safely label encoded
good_label_cols = [col for col in object_cols if 
                   set(X_train[col]) == set(X_valid[col])]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
print('Categorical columns that will be label encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)
```

One-Hot Encoding 사용 시 발생할 수 있는 문제: <u>high_cardinality에 대하여 데이터 크기가 커질 수 있으</u>므로 high_cardinality 변수에 대해서만 label encoding을 하는 것을 권장
```python
# Columns that will be one-hot encoded
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]

# Columns that will be dropped from the dataset
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

print('Categorical columns that will be one-hot encoded:', low_cardinality_cols)
print('\nCategorical columns that will be dropped from the dataset:', high_cardinality_cols)
```

In [ ]:
# 1. Score from Approach 1 (Drop Categorical Variables)¶
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

In [ ]:
# 2. Score from Approach 2 (Label Encoding)¶
from sklearn.preprocessing import LabelEncoder

# Make copy to avoid changing original data 
label_X_train = X_train.copy() # ★
label_X_valid = X_valid.copy()

# Apply label encoder to each column with categorical data
label_encoder = LabelEncoder()
for col in object_cols:
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid[col] = label_encoder.transform(X_valid[col])

print("MAE from Approach 2 (Label Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

In [ ]:
# 3. Score from Approach 3 (One-Hot Encoding)
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

## Exercise (Intermediate Machine Learning: 4 of 7)

### Pipelines
**Pipelines**는 데이터 전처리와 모델링 과정을 유지할 수 있는 방법이다.
Pipeline을 사용하게 되면 얻는 이점들:
1. Clean Code: 매번 train, validation data를 추적하지 않아도 된다.
2. Fewer Bugs: 각 과정들을  잘 잊어버리지 않는다.
3. Easier to Productionize
4. More Options for Model Validation: cross-validation

Pipeline은 아래 몇가지 step 순서대로 설계할 수 있다.
> Step1: 전처리 과정을 정의하라. `ColumnTransformer`을 이용하여 서로 다른 전처리 과정을 한꺼번에 처리할 수 있다.
  
```python
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])
```

>Step2: 모델을 정의하라.

```python
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)
```

> Step3: Pipeline을 생성하고 평가하라
`Pipeline`클래스를 사용하여 전처리과정과 모델링 과정을 정의한다.

```python
from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)
```
※ Pipeline을 설계한다면, 전처리과정과 모델링을 한줄로 끝낼수 있지만, 설계하지 못한다면 imputation, one-hot encoding, model training을 모두 각각 진행해야 된다.  
특히, 다양한 데이터 변수들을 처리해야 할 때 Pipeline 설계는 필수적이다.

∴ Pipeline설계는 clean code 설계와 error를 피하기 위해 유용한 작업이다.

## Exercise (Intermediate Machine Learning: 5 of 7)

### Cross-Validation
머신러닝은 더 나은 변수 설정, 더 나은 모델을 고르기 위한 반복하는 과정이다.

모델의 성능 검증을 위해 validation data를 나누는 데 이렇게 나눈 validation마저 정말 타당한 validation인지 확신을 갖기 위해 **cross-validation**을 써야 된다.

**Cross-Validation**에선 보통 5개 구간으로 나눈 5-folds로서 사용된다.

### When use cross-validation?
구간을 나눈 만큼 모델 검증에 시간이 많이 걸리기 때문에 비교적 작은 dataset일 경우에 사용한다.

※ `cross_val_score()`함수는 음수값을 return 해주기때문에 반드시 -1을 곱해 줘야 우리가 알고 있는 MAE 값을 구할 수 있다.

```python
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)
```

## Exercise (Intermediate Machine Learning: 6 of 7)

ensemble: 조화, 통일  
ensemble method [앙상블 모델]: 머신러닝에서도 "여러 모델이 동일한 문제를 해결하고 더 나은 결과를 얻도록 훈련시키는 기계 학습 패러다임"을 기반으로 만들어진 모델

보통 하나의 모델로 원하는 성능을 낼 수 없을 때 앙상블 학습을 사용하며, <u>개별로 학습한 여러 모델을 조합하여 일반화할 경우 성능을 향상</u>시킬 수 있다.

### Gradient Boosting
반복적으로 앙상블에 모델을 추가하는 순환적인 방법
![](https://i.imgur.com/MvCGENh.png)
그림처럼 순환이 반복된다고 보면 된다.
1. 모든 모델로부터 얻은 prediction을 현재 앙상블 prediction에 추가
2. loss를 계산한다
3. 새로운 모델에 학습시킨다 (이때 gradient의 의미는 loss를 점점 줄이는 모델을 찾는다는 의미)
4. 새로운 모델을 앙상블에 추가한다.

### XGBoost
**XGBoost**는 extreme gradient boosting의 약자이다. (성능과 속도에 초점을 둔 모델)

코드와 함께 보자면...
```python
from xgboost import XGBRegressor

my_model = XGBRegressor()
my_model.fit(X_train, y_train)
```
```python
from sklearn.metrics import mean_absolute_error

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))
```
```python
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)
```
※ early_stopping_rounds:  자동적으로 이상치를 찾음


## Exercise (Intermediate Machine Learning: 6 of 7)

### Data Leakage
training data 외의 외부 데이터가 모델을 만드는 데 사용되는 것  
따라서 잘못된 정보로 인해 정확도가 높아보일 수 있는 오류를 발생시킬 수 있다.

#### target leakage
예측을 하는 시점에서 사용불가능한 데이터가 포함될 때 발생. 이럴 경우 <u>그런 변수들을 배제 시켜야 한다.</u>

#### train-set contamination
서로 영향을 미칠 수 있는 변수들 배제.
```python
potential_leaks = ['expenditure', 'share', 'active', 'majorcards']
X2 = X.drop(potential_leaks, axis=1)

# Evaluate the model with leaky predictors removed
cv_scores = cross_val_score(my_pipeline, X2, y, 
                            cv=5,
                            scoring='accuracy')

print("Cross-val accuracy: %f" % cv_scores.mean())
```